In [1]:
import pandas as pd
import datetime

In [2]:
df_raw = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Union City\2017\Union City Transit_Data File_Final_Standardized.csv')

In [3]:
df_raw['first_route_before_survey_board'] = df_raw['1_system_before'] + '___' + df_raw['1_route_before']
df_raw['second_route_before_survey_board'] = df_raw['2_system_before'] + '___' + df_raw['2_route_before']
df_raw['first_route_after_survey_alight'] = df_raw['1_after_system'] + '___' + df_raw['1_route_after_system']
df_raw['second_route_after_survey_alight'] = df_raw['2_after_system'] + '___' + df_raw['2_route_after_system']

In [4]:
#import datetime
df_raw['survey_starttime'] = df_raw['sys_starttime'].apply(lambda x: x.split(" - ")[1])
df_raw['survey_starttime'] = df_raw['survey_starttime'].apply(lambda x: x.replace(" PDT",""))

#new_time=datetime.datetime.strptime(time,"%H:%M:%S%z")
#new_time
#df_raw['survey_endtime'] = df_raw['sys_endtime'][0].split(" - ")[1]

In [5]:
print(list(df_raw))
cols = [x.strip() for x in list(df_raw)]
df = df_raw.copy()
df.columns = cols
print(list(df))
df.to_csv(r'M:\Data\OnBoard\Data and Reports\Union City\2017\Union City Transit_fix_error_add_time_route_NO POUND OR SINGLE QUOTE.csv')

['id', 'sys_starttime', 'sys_endtime', 'run', 'route', 'rcode', 'day', 'date', 'dtype', 'strata', 'wcode', 'weight', 'alt_weight', 'interview_language', 'orig_purp', 'dest_purp', 'sch', 'school_name', 'college_name', 'access_mode', 'egress_mode', 'startlat', 'startlon', 'first_board_lat ', 'first_board_lon', 'last_alight_lat ', 'last_alight_lon', 'endlat', 'endlon', 'xfers_before', '1_system_before', '1_route_before', '1_before_lat_start', '1_before_long_start', '1_before_lat_end', '1_before_long_end', '2_system_before', '2_route_before', '2_before_lat_start', '2_before_long_start', '2_before_lat_end', '2_before_long_end', 'route.1', 'routeboard_lat', 'routeboard_long', 'routealight_lat', 'routealight_long', 'xfers_after', '1_after_system', '1_route_after_system', '1_after_lat_start', '1_after_long_start', '1_after_lat_end', '1_after_long_end', '2_after_system', '2_route_after_system', '2_after_lat_start', '2_after_long_start', '2_after_lat_end', '2_after_long_end', 'fare', 'farecat', 

In [6]:
routes = pd.DataFrame(columns = ['route'])
for i in ['first_route_before_survey_board', 'second_route_before_survey_board',
          'first_route_after_survey_alight', 'second_route_after_survey_alight']:
    route_unique = df_raw[[i]]
    route_unique.columns = ['route']
    routes = pd.concat([routes, route_unique])

routes_clean = routes.loc[routes.route.notnull()]
routes_clean.drop_duplicates(inplace=True)
print(routes_clean.shape)
routes_clean.to_csv(r'M:\Data\OnBoard\Data and Reports\Union City\2017\all_routes_raw.csv', index=False)

(30, 1)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [7]:
# bring in standard dictionary to check field consistency

# dictionary for Union City survey
var = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Union City\2017\variable_dictionary.csv',
                  encoding = "ISO-8859-1", engine='python')

# standard dictionary
var_standard = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\make-uniform\production\Dictionary for Standard Database.csv')
var_standard.columns = [x+'_s' for x in list(var_standard)]

# merge
var_merge = var.merge(var_standard, left_on='Generic_Variable', right_on='Generic_Variable_s', how='outer')

# check if 'Generic_Variable' in Union City dictionary matches the standard 'Generic_Variable'. chk1 should be empty
chk1 = var_merge.loc[(var_merge.Generic_Variable.notnull()) & (var_merge.Generic_Variable_s.isnull())]
print('Generic_Variable that should not exit:')
print(chk1.Generic_Variable.unique())
print()

# check if columns names in survey data matches 'Survey_Variable' in Union City dictionary.
# the following loops should not include variables that are needed for standardization

for i in var.loc[var.Generic_Variable.notnull()]['Survey_Variable']:
    if i not in list(df):
        print(i)
        
for i in list(df):
    if i not in list(var.Survey_Variable):
        print(i)

Generic_Variable that should not exit:
['work_lat' 'work_lon']

run
rcode
day
school_name
college_name
xfers_before
1_system_before
1_route_before
1_before_lat_start
1_before_long_start
1_before_lat_end
1_before_long_end
2_system_before
2_route_before
2_before_lat_start
2_before_long_start
2_before_lat_end
2_before_long_end
route.1
xfers_after
1_after_system
1_route_after_system
1_after_lat_start
1_after_long_start
1_after_lat_end
1_after_long_end
2_after_system
2_route_after_system
2_after_lat_start
2_after_long_start
2_after_lat_end
2_after_long_end
resident_status_county


In [8]:
# check if all the values in the survey data are included in TriDelta dictionary
# look at non-categorical variables; "diff" should be empty or only contains nan

var_clean = var[['operator', 'Survey_year', 'Survey_Variable', 'Survey_Response', 
                 'Generic_Variable', 'Generic_Response']].drop_duplicates()
var_clean = var_clean.loc[var_clean.Generic_Variable.notnull()]

for i in var_clean.loc[var_clean.Survey_Response != 'NONCATEGORICAL']['Survey_Variable'].unique():
    print(i)
    df_sub = df[['id', i]]
    var_sub = var_clean.loc[var_clean.Survey_Variable == i]

    if i in ['race_dmy_asn', 'race_dmy_blk', 'race_dmy_hisp', 'race_dmy_hwi', 'race_dmy_ind', 'race_dmy_wht']:
        var_sub.Survey_Response = var_sub.Survey_Response.apply(lambda x: int(x))
        
    compare = df_sub.merge(var_sub, left_on=i, right_on='Survey_Response', how='left')
    diff = compare.loc[compare.Generic_Response.isnull()]
    if diff.shape[0] > 0:
        print(diff[i].unique())

access_mode
at_school_after_dest_purp
[nan]
at_school_prior_to_orig_purp
[nan]
at_work_after_dest_purp
[nan]
at_work_prior_to_orig_purp
[nan]
cars
depart_hour
dest_purp
dtype
egress_mode
eng_proficient
[nan]
fare
farecat
gender
hh
hhwork
hisp
income
interview_language
langhh
[nan]
language_at_home_binary
mode
orig_purp
race_dmy_asn
[nan]
race_dmy_blk
[nan]
race_dmy_hwi
[nan]
race_dmy_ind
[nan]
race_dmy_wht
[nan]
race_other
[nan]
return_hour
sch
[nan]
strata
work_status


C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
# finally, check all necessary fields are included, and export
print(var_clean.Generic_Variable.unique())
var_clean.to_csv(r'M:\Data\OnBoard\Data and Reports\Union City\2017\vars_for_standard_dictionary.csv', index=False)

['access_mode' 'alt_weight' 'at_school_after_dest_purp'
 'at_school_prior_to_orig_purp' 'at_work_after_dest_purp'
 'at_work_prior_to_orig_purp' 'vehicles' 'date_string' 'depart_hour'
 'dest_purp' 'weekpart' 'egress_mode' 'dest_lat' 'dest_lon'
 'eng_proficient' 'fare_medium' 'fare_category' 'first_board_lat '
 'first_board_lon' 'first_route_after_survey_alight'
 'first_route_before_survey_board' 'gender' 'persons' 'workers' 'hispanic'
 'home_lat' 'home_lon' 'ID' 'household_income' 'interview_language'
 'language_at_home_detail' 'language_at_home_binary' 'last_alight_lat '
 'last_alight_lon' 'survey_type' 'orig_purp' 'race_dmy_asn' 'race_dmy_blk'
 'race_dmy_hwi' 'race_dmy_ind' 'race_dmy_wht' 'race_other_string'
 'return_hour' 'route' 'survey_alight_lat' 'survey_alight_lon'
 'survey_board_lat' 'survey_board_lon' 'student_status' 'school_lat'
 'school_lon' 'second_route_after_survey_alight'
 'second_route_before_survey_board' 'orig_lat' 'orig_lon' 'strata'
 'time_string' 'weight' 'work_sta